# Facilities

In [1]:
import pandas as pd 
import geopandas as gpd
import contextily as ctx
import folium

In [2]:
print(gpd.list_layers('data/publicfacilities.zip'))

               name geometry_type
0  publicfacilities         Point


In [3]:
facilities = gpd.read_file('data/publicfacilities.zip')

In [4]:
facilities.iloc[0]

OBJECTID                                                71
FACILITYTY                                      OPERATIONS
FACILITYNA                  ALVARADO TRANSPORTATION CENTER
ADDRESS                                      100 1ST ST SW
JURISDICTI                                            CITY
created_us                                            None
created_da                                             NaT
last_edite                                          AGSJMR
last_edi_1                             2017-03-27 00:00:00
geometry      POINT (1521422.072828997 1485835.7277358249)
Name: 0, dtype: object

In [5]:
facilities.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OBJECTID    236 non-null    int64         
 1   FACILITYTY  236 non-null    object        
 2   FACILITYNA  236 non-null    object        
 3   ADDRESS     236 non-null    object        
 4   JURISDICTI  236 non-null    object        
 5   created_us  11 non-null     object        
 6   created_da  11 non-null     datetime64[ms]
 7   last_edite  236 non-null    object        
 8   last_edi_1  236 non-null    datetime64[ms]
 9   geometry    236 non-null    geometry      
dtypes: datetime64[ms](2), geometry(1), int64(1), object(6)
memory usage: 18.6+ KB


In [6]:
facilities = facilities[['OBJECTID', 'FACILITYTY', 'FACILITYNA', 'ADDRESS', 
                           'JURISDICTI', 'geometry']]

In [7]:
facilities = facilities.rename(columns={
    'FACILITYTY': 'Type', 
    'FACILITYNA': 'Name', 
    'ADDRESS': 'Address', 
    'JURISDICTI': 'Jurisdiction'
})

In [8]:
facilities.crs

<Projected CRS: PROJCS["NAD83(HARN) / New Mexico Central (ftUS)",G ...>
Name: NAD83(HARN) / New Mexico Central (ftUS)
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
facilities = facilities.to_crs(4267)

In [10]:
def style_function(feature): 
    return {
        'color': 'transparent', 
        'fillColor': 'transparent'
    }

In [11]:
tooltip = folium.features.GeoJsonTooltip(
    fields=('Name', 'Type', 'Address','Jurisdiction'), 
    aliases=("Name:", "Type:", "Address:", "Jurisdiction:")
)

In [12]:
interactive_map = folium.Map(
    location=(35.08, -106.65), 
    zoom_start=11.3, 
    control_scale=True, 
    tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}",
    attr="Google maps",
)
addresses_layer = folium.features.GeoJson(
    facilities, 
    name='Public facilities'
)
addresses_layer.add_to(interactive_map)

tooltip_layer = folium.features.GeoJson(
    facilities, 
    style_function=style_function, 
    tooltip=tooltip
)
tooltip_layer.add_to(interactive_map)

interactive_map

In [13]:
facilities.to_file('../../../data/local/abq_city_data.gpkg', 
                  layer='PublicFacilities')

In [14]:
facilities.to_file('../../../localData/abq_city_data.gpkg', layer='PublicFacilities')